## Импортируем библиотеки

In [ ]:
import pandas as pd  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from bs4 import BeautifulSoup   
import re 
import nltk

## Загружаем данные

In [ ]:
train = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3 )

## Функция по фильтрации текстов


In [ ]:
def review_to_words( raw_review ):
    source_text = BeautifulSoup(raw_review).get_text()      
    letters_only = re.sub("[^a-zA-Z]", " ", source_text) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   
    
    return(" ".join(meaningful_words))   

## Обработка текстов

In [ ]:
reviews_count = train["review"].size
clean_reviews = []
for i in range(0, reviews_count):
    clean_reviews.append(review_to_words(train["review"][i]))

## Получим вектора признаков

In [ ]:
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 25000) 

train_data_features = vectorizer.fit_transform(clean_reviews)
train_data_features = train_data_features.toarray()

## Используем RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators = 145) 
forest = forest.fit(train_data_features, train["sentiment"])

## Получение и вывод результата

In [ ]:
clean_test_reviews = [] 
reviews_count = len(test["review"])

for i in range(0, reviews_count):
    clean_review = review_to_words( test["review"][i] )
    clean_test_reviews.append(clean_review)

test_data_feature = vectorizer.transform(clean_test_reviews)
test_data_feature = test_data_feature.toarray()

res = forest.predict(test_data_feature)

output = pd.DataFrame(data={"id":test["id"], "sentiment":res})
output.to_csv('submission.csv', index=False, quoting=3)